Importing all Libraries needed to train as well as get information from the data base

In [43]:
import pandas as pd
import logging
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)
import multiprocessing
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import pytds
create_phrases = False

Connecting to database to retrieve student and training resumes

In [44]:
connect_string = ""
try:
    conn = pytds.connect(server = '34.69.166.47', database = 'TBDraft', user = 'sqlserver', password='TBuilder')
    print("{c} is working".format(c=connect_string))
except pytds.Error as ex:
    print("{c} is not working".format(c=connect_string))

INFO - 22:46:22: Opening socket to 34.69.166.47:1433
INFO - 22:46:22: Sending PRELOGIN lib_ver=10b0000 enc_flag=2 inst_name=b'MSSQLServer' mars=False
INFO - 22:46:22: Got PRELOGIN response crypt=2 mars=0
INFO - 22:46:22: Sending LOGIN tds_ver=74000004 bufsz=4096 pid=11556 opt1=f0 opt2=2 opt3=8 cli_tz=-360 cli_lcid=1033 cli_host=DESKTOP-5D53OF5 lang= db=TBDraft
INFO - 22:46:22: switched to database TBDraft
INFO - 22:46:22: switched collation to Collation(lcid=1033, sort_id=52, ignore_case=True, ignore_accent=False, ignore_width=True, ignore_kana=True, binary=False, binary2=False, version=0)
INFO - 22:46:22: switched language to us_english
INFO - 22:46:22: Got LOGINACK tds_ver=74000004 srv_name=Microsoft SQL Server   srv_ver=f001052
INFO - 22:46:22: Sending BEGIN TRAN il=0


 is working


In [53]:

##After connecting to database extract preprocessed information
#Read in ID and resumes using pandas

#This is how to create a two column list using two lists newlist = list(zip(numbers,letters))
#Get data from server
student_id = []
student_resume = []
cursor = conn.cursor()
# cursor.execute("SELECT ALL * FROM StudentInformation") #StudentInformation is the name of the table
# for row in cursor:
#         student_id.append(row.StudentName)
#         student_resume(row.Resume)
# #Store data in pandas data frame
student_info = list(zip(student_id, student_resume))
df = pd.DataFrame(student_info, columns=['ID', 'Processed Resume'])
# df.head()
cursor.execute("SELECT ALL * FROM TrainingResumes")
result = cursor.fetchall()
for row in result:
    print(row)
    print('\n')
cursor.description
df2 = pd.read_sql_query("SELECT ALL * FROM TrainingResumes", conn)
print("Training Resumes Table\n", df2, "\n")
df3 = pd.read_sql_query("SELECT ALL * FROM StudentInformation", conn)
print("Student Information Table\n", df3)

INFO - 00:11:38: Sending query SELECT ALL * FROM TrainingResumes
INFO - 00:11:38: Sending query SELECT ALL * FROM TrainingResumes
INFO - 00:11:38: Sending query SELECT ALL * FROM StudentInformation


Training Resumes Table
 Empty DataFrame
Columns: [keyword, dataText]
Index: [] 

Student Information Table
    ID StudentName  GPA              Email  \
0   1    Student1  3.1  Student1@tamu.edu   
1   2    Student2  3.2  Student2@tamu.edu   
2   3    Student3  3.3  Student3@tamu.edu   
3   4    Student4  3.4  Student4@tamu.edu   
4   5    Student5  3.5  Student5@tamu.edu   

                                       StudentResume          ProcessedData  \
0  b'%PDF-1.7\r\n%\xb5\xb5\xb5\xb5\r\n1 0 obj\r\n...  Resume in text format   
1  b'%PDF-1.4\n%\xf6\xe4\xfc\xdf\n1 0 obj\n<<\n/T...  Resume in text format   
2  b'%PDF-1.4\n%\xf6\xe4\xfc\xdf\n1 0 obj\n<<\n/T...  Resume in text format   
3  b'%PDF-1.4\n%\xf6\xe4\xfc\xdf\n1 0 obj\n<<\n/T...  Resume in text format   
4  b'%PDF-1.4\n%\xf6\xe4\xfc\xdf\n1 0 obj\n<<\n/T...  Resume in text format   

      PhoneNumber  
0  (111) 111-1111  
1  (222) 222-2222  
2  (333) 333-3333  
3  (444) 444-4444  
4  (555) 555-5555  


Here we can either create phrases or not it would all depend on the performance

In [40]:
if(create_phrases):
    from gensim.models.phrases import Phraser, Phrases
    sent = [row.split() for row in df['clean']] #Change change ['clean'] to whatever we actually label the columns as
    phrases = Phrases(sent, min_count=5, progress_per=10000)
    bigram = Phraser(phrases)
    sentences = bigram[sent]
    documents = [TaggedDocument(doc, tags=[df.ID[i]]) for i, doc in enumerate(sentences)]
else:
    documents = [TaggedDocument(doc, tags=[df.ID[i]]) for i, doc in enumerate(df.ID)]



Training the model and saving it

In [42]:
model = Doc2Vec(documents, dm = 0, dbow_words=1 , alpha=0.025, vector_size=30, min_alpha = 0.025, min_count = 5)
model.save('d2v_model')

INFO - 22:45:46: collecting all words and their counts
INFO - 22:45:46: collected 0 word types and 0 unique tags from a corpus of 0 examples and 0 words
INFO - 22:45:46: Creating a fresh vocabulary
INFO - 22:45:46: Doc2Vec lifecycle event {'msg': 'effective_min_count=5 retains 0 unique words (0.0%% of original 0, drops 0)', 'datetime': '2021-11-22T22:45:46.117112', 'gensim': '4.1.2', 'python': '3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'prepare_vocab'}
INFO - 22:45:46: Doc2Vec lifecycle event {'msg': 'effective_min_count=5 leaves 0 word corpus (0.0%% of original 0, drops 0)', 'datetime': '2021-11-22T22:45:46.117581', 'gensim': '4.1.2', 'python': '3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'prepare_vocab'}
INFO - 22:45:46: deleting the raw counts dictionary of 0 items
INFO - 22:45:46: sample=0.001 downsamples 0 most-common words
INFO - 

RuntimeError: you must first build vocabulary before training the model